# Fisher matrix for N-counts (by Domenico Sapone): SOAPFish-release
(June 2020)

## Import libraries and ... stuff

In [1]:
# Useful packages

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.gridspec as gridspec
import math
from math import *
import scipy.special as ssp
import scipy.stats as stats

import time

import scipy 
import pandas as pd
from scipy import interpolate
from scipy.misc import derivative
import scipy.integrate as integrate

from functools import reduce #python 3
from numpy.linalg import inv
import itertools
from scipy.interpolate import interp1d

import pickle

# Class

from classy import Class

# Camb

import camb
from camb import model, initialpower

# Additional 

interp_type = 'cubic'
pow = np.power


ModuleNotFoundError: No module named 'classy'

Libreria Pickle es para guardar objetos (matrices gigantes, listas, lo que sea) que hayas generado en el codigo. La gracia es que no tienes que preocuparte de hacerle un reshape o algo para que encaje en cierto formato (por ej tipo tabla para un csv). Simplemente se guarda en la memoria tal cual esta y ya.

## Cosmology

In [ ]:
[omegamh2ref,  href, omegabh2ref, nsref, sigma8ref] = [0.143648,  0.67, 0.022445, 0.96, 0.852616]
[omegamref, omegabref] = [omegamh2ref/href**2, omegabh2ref/href**2]
[massivenus, onu, Neff] = [1, 0.00143717, 2.046]

href2 = pow(href,2)

omeganh2ref = onu*href2
omegach2ref = omegamh2ref - omegabh2ref - omeganh2ref;

#this is in km/sec
clight = 299792.458;

# steps for derivatives, for H and Da is larger
eps = 0.01

cosmpar = [omegabh2ref, href, omegamh2ref, nsref, 0, 1]
parin = ['wb','h', 'wm', 'ns','a','fz']

## Creando las especificaciones de HSC

In [ ]:
# Cosmology

def E(z,om):
    z1 = 1.0 + z
    a1 = om*(z1**3.) + (1-om)
    return np.sqrt(a1)

def f(z,om):
    return 1/E(z,om)

def rr(z,om):
    integral = scipy.integrate.quad(lambda x:f(x,om),0,z,)[0] # flat cosmology
    return 3000*integral

# we want to get kmax given the lmax=10000 (lmin = 8)

D_z = rr(0.1,omegamh2ref/href**2) # comoving distance, where 0.1 is the min redshift used

k_max = 10000/D_z
k_min = 8/D_z

print 'k = ',k_max, 'and D(z) = ',D_z, 'Mpc'
print 'k min = ',k_min

# k and z

z_1 = np.arange(0.1,1.5,0.05) # len z_1 = 28

k = np.logspace(-2, np.log10(k_max), num=500)

# creating the bins for redshift galaxy distributions

zbin = z_1[0::2]
zlist = z_1[1::2]

## CAMB: Matter power spectrum, primordial spectrum, transfer function, growth function

In [ ]:
pars = camb.CAMBparams()
pars.set_cosmology(H0=href*100, ombh2=omegabh2ref, omch2=omegach2ref)
pars.InitPower.set_params(ns=nsref)
pars.set_matter_power(redshifts=z_1, kmax=k_max)

In [ ]:
pars.NonLinear = model.NonLinear_none
results = camb.get_results(pars)
kh3,zs,Plin = results.get_linear_matter_power_spectrum(hubble_units=False, k_hunit=False)
Pk = []
for z in range(len(z_1)):
    Pk.append(Plin[z,:])

### Probando growth factor con CAMB

In [ ]:
growthfactor = results.get_redshift_evolution(k,z_1,vars=['growth'])

In [ ]:
print np.shape(growthfactor), len(k), len(z_1)

In [ ]:
growthfactor[0][-1]

La idea es compararla con CLASS que está mas abajo... independiente de si son muy parecidos o no, el punto es que debería ser independiente de k y varia un poco

In [ ]:
growthfactor[1][-1]

In [ ]:
growthfactor[167][-1]

Primordial power spectrum Fiducial

In [ ]:
trans = results.get_matter_transfer_data()
kh = trans.transfer_data[0,:,0]
k = kh*results.Params.h
primordial_PK = results.Params.scalar_power(k)

primordial = scipy.interpolate.interp1d(k,primordial_PK,kind=interp_type,fill_value='extrapolate')

Transfer function

In [ ]:
transfer_fid = []
for z in range(len(z_1)):
    transfer = trans.transfer_data[model.Transfer_tot-1,:,z]
    f = scipy.interpolate.interp1d(k,transfer,kind=interp_type,fill_value='extrapolate')
    transfer_fid.append(f)

### Growth function con CLASS

In [ ]:
a_s = 2e-09
params = {'output':'mPk','non linear':'halofit','omega_b':omegabh2ref,'omega_cdm':omegach2ref,'h':href,'A_s':a_s,'n_s':nsref,'P_k_max_1/Mpc':k_max,'z_max_pk':2.}
cosmo = Class()
cosmo.set(params)
cosmo.compute()
dz_fid = []
for z1 in z_1:
    lineargrowth = cosmo.scale_independent_growth_factor(z1)
    dz_fid.append(lineargrowth)


In [ ]:
dz_fid[0]

In [ ]:
eps=0.01
def lado(lado):
    if lado=='r':
        valor = 1+eps
    elif lado=='l':
        valor = 1-eps
    return valor

## Right

Ojo en principio k_max tambien cambia evaluando parametros a la derecha y a la izquierda, pero la verdad es que no deberia cambiar mucho y ya >20 contribuye bastante poco

In [ ]:
side1 = 'r'
cosmo1 = [omegabh2ref, href, omegach2ref, nsref]
cosmo_der = list(np.array(cosmo1)*lado(side1))

keyz = ['output', 'non linear', 'P_k_max_1/Mpc','z_max_pk']
value = ['mPkl', 'halofit', k_max, 2.]
cosm = ['omega_b','h','omega_cdm','n_s']

CLR=[]

primordial_right = []
transfer_right = []
dz_der = []
keis_r = []
dz_der = []

for i in range(len(cosmo1)):
    
    if i!=2:
        param_fid_der = cosmo1[:i]+[cosmo_der[i]]+cosmo1[i+1:]
        href1 = param_fid_der[1]
        
    elif i==2:
        param_fid_der = [omegabh2ref, href*lado(side1), omegach2ref*lado(side1) + omegabh2ref*eps, nsref]
        href1 = param_fid_der[1]
        
    dic_param_fid_der = dict(zip(cosm,param_fid_der))
    dic = dict(zip(keyz,value))
    dic.update(dic_param_fid_der)
    cosmor = Class()
    cosmor.set(dic)
    cosmor.compute()
    
    pars = camb.CAMBparams()
    
    pars.set_cosmology(H0=param_fid_der[1]*100, ombh2=param_fid_der[0], omch2=param_fid_der[2])
    pars.InitPower.set_params(ns=param_fid_der[3])
    pars.set_matter_power(redshifts=z_1, kmax=k_max)
    pars.NonLinear = model.NonLinear_none
    results = camb.get_results(pars)
    
    kh2,zs,Plin = results.get_linear_matter_power_spectrum(hubble_units=False, k_hunit=False)
    keis_r.append(kh2)
    
    trans = results.get_matter_transfer_data()

    per_param = []
    transfer_right_per_param = []
    dz_per_param = []
    
    kh = trans.transfer_data[0,:,0]
    k = kh*results.Params.h
    primordial_PK = results.Params.scalar_power(k)
    f1 = scipy.interpolate.interp1d(k,primordial_PK,kind=interp_type,fill_value='extrapolate')
    primordial_right.append(f1)
    
    for z in range(len(z_1)):

        per_param.append(Plin[z,:])
        
        transfer = trans.transfer_data[model.Transfer_tot-1,:,z]
        f = scipy.interpolate.interp1d(k,transfer,kind=interp_type,fill_value='extrapolate')
        transfer_right_per_param.append(f)
        lineargrowth = cosmo.scale_independent_growth_factor(z)
        dz_per_param.append(lineargrowth)
        
    dz_der.append(dz_per_param)
    CLR.append(per_param)   
    transfer_right.append(transfer_right_per_param)

In [ ]:
zetas=[0,27]
for i in zetas:
    plt.loglog(keis_r[0],per_param[i],c='r',label='1',linewidth=5.0)
    plt.loglog(keis_r[1],per_param[i],c='g',label='2',linewidth=3.0)
    plt.loglog(keis_r[2],per_param[i],c='b',label='3',linewidth=.5)
    plt.legend()

## Left

In [ ]:
side1 = 'l'
cosmo_izq = list(np.array(cosmo1)*lado(side1))

keyz = ['output', 'non linear', 'P_k_max_1/Mpc','z_max_pk']
value = ['mPkl', 'halofit', k_max, 2.]
cosm = ['omega_b','h','omega_cdm','n_s']

CLL=[]
primordial_left = []
transfer_left = []
dz_left = []
keis_l = []
for i in range(len(cosmo1)):
    if i!=2:
        param_fid_der = cosmo1[:i]+[cosmo_izq[i]]+cosmo1[i+1:]
        href1 = param_fid_der[1]
    elif i==2:
        param_fid_der = [omegabh2ref, href*lado(side1), omegach2ref*lado(side1) - omegabh2ref*eps, nsref]
        href1 = param_fid_der[1]
    
    dic_param_fid_der = dict(zip(cosm,param_fid_der))
    dic = dict(zip(keyz,value))
    dic.update(dic_param_fid_der)
    cosmor = Class()
    cosmor.set(dic)
    cosmor.compute()
    
    pars.set_cosmology(H0=param_fid_der[1]*100, ombh2=param_fid_der[0], omch2=param_fid_der[2])
    pars.InitPower.set_params(ns=param_fid_der[3])
    pars.set_matter_power(redshifts=z_1, kmax=k_max)
    pars.NonLinear = model.NonLinear_none
    results = camb.get_results(pars)
    
    kh2,zs,Plin = results.get_linear_matter_power_spectrum(hubble_units=False, k_hunit=False)
    keis_l.append(kh2)
    
    trans = results.get_matter_transfer_data()

    per_param = []
    transfer_left_per_param = []
    dz_per_param = []
    
    kh = trans.transfer_data[0,:,0]
    k = kh*results.Params.h
    primordial_PK = results.Params.scalar_power(k)
    f1 = scipy.interpolate.interp1d(k,primordial_PK,kind=interp_type,fill_value='extrapolate')
    primordial_left.append(f1)
    
    for z in range(len(z_1)):

        per_param.append(Plin[z,:])
        #print i, z, len(Plin[z,:]), len(kh2)
        transfer = trans.transfer_data[model.Transfer_tot-1,:,z]
        f = scipy.interpolate.interp1d(k,transfer,kind=interp_type,fill_value='extrapolate')
        transfer_left_per_param.append(f)
        
        lineargrowth = cosmo.scale_independent_growth_factor(z)
        dz_per_param.append(lineargrowth)

    transfer_left.append(transfer_left_per_param)
    CLL.append(per_param)
    dz_left.append(dz_per_param)

### Modificamos los anteriores arreglos agregando dos fiducial al final, pues esos serviran para pi=4 y pi=5

In [ ]:
for i in range(2): 
    primordial_right.append(primordial)
    primordial_left.append(primordial)
    transfer_right.append(transfer_fid)
    transfer_left.append(transfer_fid)


In [ ]:
for i in range(2):
    dz_left.append(dz_fid)
    dz_der.append(dz_fid)

In [ ]:
def primordialf(k):
    return primordial(k)

def primordialr(k,pi): # k float
    prim = primordial_right[pi]
    return prim(k)

def primordiall(k,pi):
    prim = primordial_left[pi]
    return prim(k)

def transfer(z,k,pi,side): # z index, k float
    
    if side==0 or pi==5 or pi==4:
        tf = transfer_fid[z]
    elif side==1:
        tf = transfer_right[pi][z]
    elif side==-1:
        tf = transfer_left[pi][z]
    valor = tf(k)    
    return valor


## Background quantities

In [ ]:
#Units of Mpc/h or h/Mpc it's like setting h=1 

def sp(pi, pj):
    if pi==pj: return 1
    else: return 0
    
def hubble(z, h, wm, pi, side):
    if side==1: return 100*h*(1+eps*sp(pi, 1))*np.sqrt(wm*(1+eps*sp(pi, 2))*(1+z)**3/(h*(1+eps*sp(pi, 1)))**2+ 1-wm*(1+eps*sp(pi, 2))/(h*(1+eps*sp(pi, 1)))**2)
    if side==0: return 100*h*np.sqrt(wm*(1+z)**3/(h**2) + 1-wm/(h**2))
    if side==-1: return 100*h*(1-eps*sp(pi, 1))*np.sqrt(wm*(1-eps*sp(pi, 2))*(1+z)**3/(h*(1-eps*sp(pi, 1)))**2+ 1-wm*(1-eps*sp(pi, 2))/(h*(1-eps*sp(pi, 1)))**2)



def int_dcom(z, h, wm, pi, side):
    return clight/hubble(z,h,wm, pi, side)

def dcom0(z, h, wm, pi, side):
    intdcom = scipy.integrate.quad(int_dcom,  0.,  z, args = (h, wm, pi, side))[0]
    return intdcom

dcom = np.vectorize(dcom0)

def da(z, h, wm, pi, side):
    return dcom(z, h, wm, pi, side)/(1+z)


#the volume should not change with the cosmoslogy? 
def volumesurvey(z1, z2, h, wm): #*pow(1,3)
    radtoarea = 4*np.pi/3/(4*np.pi*pow(180/np.pi,2))
    return radtoarea*(pow((1+z2)*da(z2, h, wm, 0, 0),3)-pow((1+z1)*da(z1, h, wm, 0, 0),3))
     
    


def dvol(i, h, wm):
    return volumesurvey(zbin[i], zbin[i+1], h, wm)
        

In [ ]:
int_dcom(0, href, omegamh2ref, 1, 0)

## Setting some constants useful for N-counts
think whether this should change with the cosmology or not: I'm assuming it does, but for no obvious reasons.

In [ ]:
mpckm =  3.086*10**19 #Mpc to km
kmmpc = 1/(3.086*10**19) #km to Mpc


#G is in km**3/kg/sec**2 converted in Mpc**3/kg/sec**2
Gnewton = 6.674*10**(-20.)*kmmpc**3

#clight = 299792.458 #km/sec
def hubconst(h, side):
    #this has units in 1/s
    if side==1: return 100*h*(1+eps)*kmmpc
    elif side==0: return 100*h*kmmpc
    elif side==-1: return 100*h*(1-eps)*kmmpc


def ee(z, h,  wm, pi, side):
    if side==1: return np.sqrt(wm*(1+eps*sp(pi, 2))*(1+z)**3/(h*(1+eps*sp(pi, 1)))**2+ 1-wm*(1+eps*sp(pi, 2))/(h*(1+eps*sp(pi, 1)))**2)
    if side==0: return np.sqrt(wm*(1+z)**3/(h**2) + 1-wm/(h**2))
    if side==-1: return np.sqrt(wm*(1-eps*sp(pi, 2))*(1+z)**3/(h*(1-eps*sp(pi, 1)))**2+ 1-wm*(1-eps*sp(pi, 2))/(h*(1-eps*sp(pi, 1)))**2)


#scale depending on the mass: it has to be in Mpc as k has units in 1/Mpc
def rm(m, z, h, wm, pi, side):
    return (2*Gnewton*m/(hubconst(h, side)*ee(z, h, wm, pi, side))**2)**(1/3.)


def rhoc(h, side):
    
    #background density of the universe in kg/Mpc**3
    return 3*hubconst(h, side)**2/(8*np.pi*Gnewton)


def rhob(z, h, wm, pi, side):
    if side==1: return rhoc(h, side)*wm*(1+eps*sp(pi, 2))/(h*(1+eps*sp(pi, 1)))**2
    elif side==0: return rhoc(h, side)*wm/h**2
    elif side==-1: return rhoc(h, side)*wm*(1-eps*sp(pi, 2))/(h*(1-eps*sp(pi, 1)))**2



In [ ]:
rm(10**45., 0.1, 0.67, 0.14, 1, 0)

In [ ]:
ee(0.1, 0.67, 0.14, 5, 1)

## Reading power spectra from CAMB

In [ ]:

#Call the Pks and make matrices with interpolated files: faster because info is stored 

# This I need to change from units in h/Mpc to 1/Mpc
def hsr(pi):
    if pi ==1: return href*(1+eps)
    else: return href

def hsl(pi): ######## aqui cambie porque decia pi==1
    if pi ==-1: return href*(1-eps)
    else: return href
    
    
#k = np.logspace(-2, np.log10(k_max), num=217)
vecintpk = []
for zi in range(len(zlist)):
    pks=Pk[zi]
    interpolpk = scipy.interpolate.interp1d(np.log(kh3),np.log(pks),kind=interp_type,fill_value='extrapolate')
    vecintpk.append(interpolpk)

vecintpkr = []
cantidad = len(zlist)
for zi in range(cantidad):
    vecintpkr.append([])
    for pi in range(len(cosmo1)):
        k = keis_r[pi]
        cls=CLR[pi]
        pksr=cls[zi]
        vecintpkr[zi].append(scipy.interpolate.interp1d(np.log(k),np.log(pksr),kind=interp_type,fill_value='extrapolate'))

vecintpkl = []
for zi in range(cantidad):
    vecintpkl.append([])
    for pi in range(len(cosmo1)):
        k = keis_l[pi]
        cls=CLL[pi]
        pksl=cls[zi]
        #print pi,len(pksl), len(kh2), len(pksr)
        vecintpkl[zi].append(scipy.interpolate.interp1d(np.log(k),np.log(pksl),kind=interp_type,fill_value='extrapolate'))
   



In [ ]:


#interpolation of the fiducial Pks in k
def intpks(zi,k): #DS Takes true k (not an index)
    return np.exp(vecintpk[zi](np.log(k)))

#interpolation of the fiducial log Pks in k
def intlogpks(zi,k): 
    return vecintpk[zi](np.log(k))


#interpolation of the right Pks in k
def intpksr(zi,k, pi): 
    return np.exp(vecintpkr[zi][pi](np.log(k)))

#interpolation of the right log Pks in k
def intlogpksr(zi,k,pi): 
    return vecintpkr[zi][pi](np.log(k))


#interpolation of the left Pks in k
def intpksl(zi, k, pi): 
    return np.exp(vecintpkl[zi][pi](np.log(k)))


#interpolation of the left log Pks in k                  
def intlogpksl(zi,k,pi): 
    return vecintpkl[zi][pi](np.log(k))



# Testing the Pk evaluated by CAMB and the growth factor defined

# Evaluating the rms density fluctuations 

\begin{equation}
        \sigma^2(M,z)= \frac{1}{2\pi^2}\int_{k_{\rm m}}^{k_{\rm M}} k^2 P(k,z)W^2(k\,R(M)){\rm d}k
\end{equation}
we do not use the growth factor $G(z)$ but we let the $P(k)$ vary with $z$ and 
\begin{eqnarray}
        W(x) &=& \frac{3}{x^3}\left(\sin(x)-x\,\cos(x)\right)\\
        R(M) &=& \left(\frac{3 M}{4\pi \rho_b(z)}\right)^{1/3}\\
\end{eqnarray}
and a new variable was defined to be
\begin{equation}
\nu = \delta_c/\sigma(M,z)
\end{equation}
being $\delta_c = 1.686$

In [ ]:
min_k = min(kh3)
min_k

In [ ]:
#defining the top-hat filter
def window(x):
    return 3.0*(np.sin(x)-x*np.cos(x))/x**3.


kminint = min(kh3)
kmaxint = k_max


#For the moment I don't need the growth factor because the Pk have been evaluated at different z
# which means they've been properly evaluated
def intwind(k, zi,  m, h, wm, pi, side):
    
    if (side==1 and pi==4) or (side==1 and pi==5):
        return k**2*intpks(zi,k)*window(k*rm(m, zlist[zi], h, wm, pi, side))**2
    
    elif (side==-1 and pi==4) or (side==-1 and pi==5):
        return k**2*intpks(zi,k)*window(k*rm(m, zlist[zi], h, wm, pi, side))**2
    
    elif side==1:
        return k**2*intpksr(zi,k, pi)*window(k*rm(m, zlist[zi], h, wm, pi, side))**2
    
    elif side == 0: 
        #print 'rpks',intpks(zi,k)
        return k**2*intpks(zi,k)*window(k*rm(m, zlist[zi], h, wm, pi, side))**2
    
    elif side == -1: return k**2*intpksl(zi,k, pi)*window(k*rm(m, zlist[zi], h, wm, pi, side))**2
    


def smfunc(zi, m, h, wm, pi, side):
    kgrid = np.linspace(np.log(kminint), np.log(kmaxint), 2**8+1)
    dk = np.diff(kgrid)[0]
    intwind1 = intwind(np.exp(kgrid), zi,  m, h, wm, pi, side)*np.exp(kgrid)
    #print intwind1
    #I1 = np.cumsum(dk*intwind1)
    I1 =  scipy.integrate.romb(intwind1, dx=dk)
    return np.sqrt(I1/(2*pow(np.pi,2)))



def lnsmfunc(zi, m, h, wm, pi, side):
    kgrid = np.linspace(np.log(kminint), np.log(kmaxint), 2**8+1)
    dk = np.diff(kgrid)[0]
    intwind1 = intwind(np.exp(kgrid), zi,  m, h, wm, pi, side)*np.exp(kgrid)
    I1 =  scipy.integrate.romb(intwind1, dx=dk)
    return np.log(np.sqrt(I1/(2*pow(np.pi,2))))





# nu = log(d_c/sigma)
def invlnsmfunc(zi, m, h, wm, pi, side):
    dc = 1.686
    return np.log(dc/smfunc(zi, m, h, wm, pi, side))



# dln\sigma**-1/dlnM
def dlninvsdlnm(zi, m, h, wm, pi, side):
   
    dright = invlnsmfunc(zi, m**(1+eps), h, wm, pi, side)
    dleft = invlnsmfunc(zi, m**(1-eps), h, wm, pi, side)
    return (dright-dleft)/(2*eps*np.log(np.float(m)))
 
    

# dln\sigma**-1/dM
def dlninvsdm(zi, m, h, wm, pi, side):
   
    dright = invlnsmfunc(zi, m*(1+eps), h, wm, pi, side)
    dleft = invlnsmfunc(zi, m*(1-eps), h, wm, pi, side)
    return (dright-dleft)/(2*eps*m)
    
    
# d\nu/dM
def dnudm(zi, m, h, wm, pi, side):
    dc = 1.686
    nuright = dc/smfunc(zi, m*(1+eps), h, wm, pi, side)
    nuleft = dc/smfunc(zi, m*(1-eps), h, wm, pi, side)
   
    return (nuright-nuleft)/(2*eps*m)
    


In [ ]:
smfunc(1, 10**45., href, omegamh2ref, 1, 0)

# Some tests

In [ ]:
solar_mass = 2*10**30.
init_mass = 10**14.
fin_mass = 10**16.


expmass = np.arange(np.log10(solar_mass*init_mass), np.log10(solar_mass*fin_mass), 0.2)
v_dlninvsdlnm = []
for i in range(len(expmass)):
    v_dlninvsdlnm.append(dlninvsdlnm(1, 10**expmass[i], href, omegamh2ref, 0, 0))

# Mass function for cluster; taken from: arXiv:0005260

Sheth Thormen mass function is
\begin{equation}
\frac{dn_{\rm ST}}{d\ln
M}=\frac{\bar{\rho}}{M}\nu\frac{d\ln\nu}{d\ln M} f_{ST}(\nu) = \bar{\rho}\frac{d\nu}{d M} f_{ST}(\nu)\,,
\end{equation}
where 
\begin{equation}
f_{ST}(\nu) = A\sqrt{\frac{2a}{\pi}}\left(1+\frac{1}{(\sqrt{a}\nu)^{2q}}\right)a\exp\left(-\frac{(\sqrt{a}\nu)^2}{2}\right)
\end{equation}
with $A=0.3222$, $a=0.707$, and $q=0.3.$ 

The constants and the form of the collapse threshold are adjusted somewhat using calibrations from cosmological simulations of structure formation. For this reason, this function is a pretty accurate  representation of the mass function of collapsed halos in cosmological simulations. It is accurate to better than $30\%$ over galaxy scale masses and has larger error at large masses. 

In [ ]:
# Sheth-Tormen differential mass function     
def stfun1(zi, m, h, wm, pi, side):
    capa = 0.322
    a= 0.707
    q = 0.3
    dc = 1.686
    nu = dc/smfunc(zi, m, h, wm, pi, side)
    return capa*a*np.sqrt(2/np.pi)*(1+1/(a*nu)**(2**q))*np.exp(-(a*nu)**2/2)


#Sheth, Mo & Tormen 2001
def stfun(zi, m, h, wm, pi, side):
    capa = 0.322
    a= 0.707
    q = 0.3
    dc = 1.686
    nu = dc/smfunc(zi, m, h, wm, pi, side)
    return capa*np.sqrt(2*a/np.pi)*(1+1/(a*nu**2)**(q))*np.exp(-a/2*nu**2)



# Sheth-Tormen differential mass function to plot as a function of sigma
def stfun_toplot(zi, m, h, wm, sig):
    capa = 0.322
    a= 0.707
    q = 0.3
    dc = 1.686
    nu = dc/sig
    return capa*np.sqrt(2*a/np.pi)*(1+1/(a*nu**2)**(q))*np.exp(-(a*nu**2)/2)




#this is - dn/dM
def dndm_st(zi, m, h, wm, pi, side):
    # using Sheth and Tormen 
    # the units number/Volume but I need to express it in Mpc, hence
    # I need to convert it dividing it by kmmpc**3 factor 
    return rhob(zlist[zi], h, wm, pi, side)/m*dnudm(zi, m, h, wm, pi, side)*stfun(zi, m, h, wm, pi, side)#/kmmpc**3


In [ ]:
dndm_st(1, 10**45., href, omegamh2ref, 1, 1)

## Mass Function: primordial Non-Gaussianities

The mass function to work on is

\begin{equation}
\frac{dn}{dM}(M,z) = \frac{dn_{ST}}{dM}(M,z) \left[ 1-\frac{A}{\delta_c}\left( 1-\nu_c^2 + \frac{1}{(ln\nu_c)'}\frac{d}{dM}\right)\frac{4\pi}{V_M}\int_0^{R_M} dx x^2 sin\left(\frac{b(x)\delta_c(z)}{f_\zeta}\right)exp\left( \frac{-\sigma_W^2(x)}{2f_\zeta^2}\right)  \right]
\end{equation}

where 
\begin{equation}
\sigma^2(x) = \int_k P_\zeta(k)\left( 1-exp(ikx)W(k)\right) \qquad W(k) = \frac{3}{5}W_M(k)\alpha(k) \qquad W_M(k)=\frac{1}{V_M}H(R_M - x)
\end{equation}
and
\begin{equation}
\alpha(k) = \frac{2r_H^2k^2T(k)D(z)}{3\Omega_m} \qquad b(x) = \frac{\int_k exp(ikx) W(k)P_\zeta(k)}{\int_k W^2(k)P_\zeta(k)}
\end{equation}

Also, remember that 
\begin{equation}
\int_k = \frac{1}{2\pi^3} \int d^3k
\end{equation}

## $R_M$ and $V_M$

So first we will define the functions for $R_M = (3M/4\pi \bar{\rho})^{1/3}$ and $V_M$, where $\bar{\rho} = \Omega_m \rho_{cr}$

In [ ]:
def vm2(m, z, h, wm, pi, side):
    return (4*np.pi*(rm(m, z, h, wm, pi, side))**3.)/(3.)

## $\sigma_W^2 (x)$: W(k)

#### $W_M(k)$

In [ ]:
def window_m1(k, m, z, h, wm, pi, side):
    z2 = z_1[z]
    radius = rm(m, z2, h, wm, pi, side)

    return vm2(m, z2, h, wm, pi, side)*np.heaviside(radius-k,1)


In [ ]:
def alphak(m,k, z, h, wm, pi, side):
    
    # z debe ser indice!!!, k no debe ser index, pues en transfer se evalua k en la interpolacion
    # ojo estoy teniendo valores negativos debido a Transfer function
 
    z1 = z_1[z]
    a1 = 2.*(k**2.)*int_dcom(z1, h, wm, pi, side)**2.
    
    if side==0:
        a3 = wm/(h**2)
        alph = dz_fid[z]
 
    elif side==1: 
        a3 = wm*(1+eps*sp(pi, 2))/(h*(1+eps*sp(pi, 1)))**2
        alph = dz_der[pi][z] # dz es indep de escala
        
    elif side==-1: 
        a3 = wm*(1-eps*sp(pi, 2))/(h*(1-eps*sp(pi, 1)))**2
        alph = dz_left[pi][z]
        
    min_k = min(kh3)
    maximo = transfer(z,min_k,pi,side)
    return a1*alph*(transfer(z,k,pi,side)/maximo)/(3.*a3)

#### W(k)

In [ ]:
def window_ng(k,m, z, h, wm, pi, side): # aqui z index, en window_m1 lo pasa a float
    factor = 3/5.
    return factor*window_m1(k, m, z, h, wm, pi, side)*alphak(m,k, z, h, wm, pi, side)

#window_ng = np.vectorize(window_ng)

In [ ]:

def sigmaw_and_bx(x, m, z, h, wm, pi, side):
    
    factor = 1./(2*np.pi)**3.
    kgrid = np.linspace(np.log(kminint), np.log(kmaxint), 2**8+1)
    dk = np.diff(kgrid)[0]
 
    expgrid = np.exp(kgrid)

    if side == 0 or pi==5 or pi==4:
        primord = primordialf(np.exp(kgrid))
        dz = dz_fid[z]
        d0 = dz_fid[0]

    elif side == 1:
        primord = primordialr(np.exp(kgrid),pi)
        dz = dz_der[pi][z]
        d0 = dz_der[pi][0]
    elif side == -1:
        primord = primordiall(np.exp(kgrid),pi)
        dz = dz_left[pi][z]
        d0 = dz_left[pi][0]
        

    I1 =  scipy.integrate.romb(primord, dx=dk)

    integral1 = primord*(np.exp(1j*expgrid*x)*window_ng(expgrid,m, zi, h, wm, pi, side))
    integral2 = primord*(window_ng(expgrid,m, zi, h, wm, pi, side))**2.
    
    I2 =  scipy.integrate.romb(integral1, dx=dk)
    I3 =  scipy.integrate.romb(integral2, dx=dk)

    b_x = I2.real/I3.real
    
    s_x = (factor*(I1).real) - ((factor*(I2.real)**2.)/I3.real)

    return s_x, b_x, d0/dz


In [ ]:
def total_integral(f_zeta,z, m, h, wm, pi, side):
    
    factor = 1./(2.*np.pi**2.)
    
    factor1=4*np.pi/vm2(m, z, h, wm, pi, side)
    xgrid = np.linspace(0, rm(m, z, h, wm, pi, side), 2**8+1)
    dx1 = np.diff(xgrid)[0]

    sw, bx, d0dz = sigmaw_and_bx(xgrid, m, z, h, wm, pi, side)
    deltac = 1.686*d0dz
    
    if side==1:      
        integer1 = math.sin(bx*deltac/(f_zeta*(1+eps*sp(pi, 5))))*np.exp(-0.5*sw/(f_zeta*(1+eps*sp(pi, 5)))**2.)
    elif side==0:
        integer1 =math.sin(bx*deltac/(f_zeta))*np.exp(-0.5*sw/(f_zeta)**2.)
    elif side==-1:
        integer1 =math.sin(bx*deltac/(f_zeta*(1-eps*sp(pi, 5))))*np.exp(-0.5*sw/(f_zeta*(1-eps*sp(pi, 5)))**2.)
    
    #####################################################################################################
    
    return scipy.integrate.romb(integer1*xgrid**2, dx=dx1)

total_integral1 = np.vectorize(total_integral)

In [ ]:
start = time.time()
print total_integral(0.005, 0 , 10**44.2, href, omegamh2ref, 5, 1)
print time.time() - start

Now we need to evaluate the expression for

\begin{equation}
\sigma^2_W(x) = \int_k P_\zeta(k)\left( 1-exp(ikx)W(k)\right)
\end{equation}

For b(x):
    
\begin{equation}
b(x) = \frac{\int_k exp(ikx) W(k)P_\zeta(k)}{\int_k W^2(k)P_\zeta(k)}
\end{equation}

In [ ]:
def delta_c(zi, h, wm, pi, side):
    if side==0:
        dz = dz_fid[zi]
        d0 = dz_fid[0]

    elif side==1:
        dz = dz_der[pi][zi]
        d0 = dz_der[pi][0]
    elif side==-1:
        dz = dz_left[pi][zi]
        d0 = dz_left[pi][0]
        
    return 1.686*d0/dz


In [ ]:
def sigmx(m, z, h, wm, pi, side):
    
    factor = 1./(2*np.pi)**3.
    kgrid = np.linspace(np.log(kminint), np.log(kmaxint), 2**8+1)
    dk = np.diff(kgrid)[0]
    
    if side == 0 or pi==5 or pi==4:
        primord = primordialf(np.exp(kgrid))
    elif side == 1:
        primord = primordialr(np.exp(kgrid),pi)
    elif side == -1:
        primord = primordiall(np.exp(kgrid),pi)
        
    integral2 = primord*pow(window_ng(np.exp(kgrid),m, z, h, wm, pi, side),2)
    I2 =  scipy.integrate.romb(integral2, dx=dk)
    return np.sqrt(factor*I2)

In [ ]:
start = time.time()
print sigmx(10**45., 1, 0.67, 0.14, 5, -1)
print time.time() - start

In [ ]:
def dlnnucdm(m, z, h, wm, pi, side):
    eps=0.01
    dc=delta_c(z, h, wm, pi, side)
    nucr = dc/sigmx(m*(1+eps), z, h, wm, pi, side)
    nucl = dc/sigmx(m*(1-eps), z, h, wm, pi, side)
    
    return (nucr-nucl)/(2*eps*m)

In [ ]:
def dintegraldm(f_zeta, z, m, h, wm, pi, side):
    eps=0.01

    didmr = total_integral(f_zeta, z, m*(1+eps), h, wm, pi, side)
    didml = total_integral(f_zeta, z, m*(1-eps), h, wm, pi, side)
    return (didmr-didml)/(2.*eps*m)

### Z index!!

In [ ]:
def non_gaussian(A,f_zeta,z, m, h, wm, pi, side):
    
    if side==1:
        A1 = A*(1+eps*sp(pi,4))
    elif side==0:
        A1 = A
    elif side==-1:
        A1 = A*(1-eps*sp(pi,4))
        
    dc=delta_c(z, h, wm, pi, side)
    nu = dc/sigmx(m, z, h, wm, pi, side)
    parentesis = 1 - nu**2 + (1/dlnnucdm(m, z, h, wm, pi, side))*dintegraldm(f_zeta, z, m, h, wm, pi, side)
    
    return 1-(A1/dc)*parentesis

#non_gaussian = np.vectorize(non_gaussian)

In [ ]:
start = time.time()
print non_gaussian(0.01,1., 10, 10**45., 0.67, 0.14, 5, -1)
print time.time() - start

# Final Mass Function

In [ ]:
def halomassfunction(A,f_zeta,zi, m, h, wm, pi, side):
    if A==0: # LCDM
        valor = dndm_st(zi, m, h, wm, pi, side)
    else:
        valor = dndm_st(zi, m, h, wm, pi, side)*non_gaussian(A,f_zeta,zi, m, h, wm, pi, side)
    return valor


In [ ]:
start = time.time()
print halomassfunction(0.001,0.05,1, 10**45., href,  omegamh2ref, 1, 0)
print time.time() - start

# Average number density of clusters

\begin{equation}
n_i = \int_{z_{min}}^{z_{max}}{\rm d}z\frac{\chi^2(z)}{H(z)}\int_{M_{\rm min}}^{M_{\rm max}} \frac{{\rm d}M}{M}\frac{{\rm d}n}{{\rm d}\ln M}\times
    \frac{1}{2} \left[erfc(x_i)-erfc(x_{i+1})\right]\theta(z-z_{min})\theta(z_{max}-z)
\end{equation}
or 
\begin{equation}
n_i = \int_{z_{min}}^{z_{max}}{\rm d}z\frac{\chi^2(z)}{H(z)}\int_{M_{\rm min}}^{M_{\rm max}} {\rm d}M\frac{{\rm d}n}{{\rm d}M}\times
    \frac{1}{2} \left[erfc(x_i)-erfc(x_{i+1})\right]\theta(z-z_{min})\theta(z_{max}-z)
\end{equation}
where $erfc$ is the complementary error function, and 
\begin{equation}
x\left(M^{obs}\right) = \frac{\ln M^{obs}-\ln M - \ln M^{bias}}{\sqrt{2\sigma_{\ln M}^{2}}}
\end{equation}
Assumptions: $\ln M^{bias}=0$ and $\sigma_{\ln M}= 0.3$
We also define, for sake of simplicity, the quantity
\begin{equation}
S_{i(b)}(z) = \frac{1}{2} \left[erfc(x_i)-erfc(x_{i+1})\right]\theta(z-z_{min})\theta(z_{max}-z)
\end{equation}
which is a sort of a filter in z bin and Mass bin

In [ ]:
from scipy import special

def x(mobs, m):
    sigmam = 0.3
    return (np.log(mobs)-np.log(m))/np.sqrt(2*sigmam**2)


def integnmean1(m, z, mobs, zi, h, wm, pi, side,A, f_zeta):
    sigmam = 0.3
    heavy = np.heaviside(z-zbin[zi], 1)*np.heaviside(zbin[zi+1]-z, 1)
    erfi = (special.erfc(x(mobs, m))-special.erfc(x(mobs*(1+np.exp(sigmam)), m)))*1/2
    vol_term = clight*dcom(z, h, wm, pi, side)**2/hubble(z, h, wm, pi, side)
    return vol_term*heavy*erfi*halomassfunction(A,f_zeta,zi, m, h, wm, pi, side)

integnmean = np.vectorize(integnmean1)


def integrate2D(dfun, zgrid, lellgrid):
    
    muint = [scipy.integrate.romb(dfun.T[i], dx=np.diff(lellgrid)[0]) for i in range(zgrid.size)]
    return scipy.integrate.romb(muint, dx=np.diff(zgrid)[0])


#average number density of clusters
# much faster, it needs since it has to be done in a small window, 
# we can choose small number of subdivision, error of 1e-4% with respect to the full integration

def nmean1(A, f_zeta,zi, mobs, h, wm, pi, side, delm):
    mminint = mobs*10**(-delm)
    mmaxint = mobs*10**(+delm)
    
    zmin = zbin[zi]
    zmax = zbin[zi+1]
  
    
    zgrid = np.linspace(zmin,zmax, 2**2+1) 
    massgrid = np.linspace(np.log10(mminint), np.log10(mmaxint), 2**3+1)
    m, z = np.meshgrid(massgrid, zgrid)
    return integrate2D(np.log(10)*10**m*integnmean(10**m, z, mobs, zi, h, wm, pi, side,A, f_zeta), massgrid, zgrid)

nmean = np.vectorize(nmean1)


## New Halo Bias

In [ ]:
def i3(f_zeta, m, z, h, wm, pi, side):
    factor = 1./(2.*np.pi**2.)

    factor1=4*np.pi/vm2(m, z, h, wm, pi, side)
    xgrid = np.linspace(0, rm(m, z, h, wm, pi, side), 2**8+1)
    dx1 = np.diff(xgrid)[0]

    
    sw, bx, d0dz = sigmaw_and_bx(xgrid, m, z, h, wm, pi, side)
    deltac = 1.686*d0dz
    
    ########################################################
    
    if side==1:      
        integer1 = math.cos(bx*deltac/(f_zeta*(1+eps*sp(pi, 5))))*np.exp(-0.5*sw/(f_zeta*(1+eps*sp(pi, 5)))**2.)
        fz = f_zeta*(1+eps*sp(pi, 5))
    elif side==0:
        integer1 =math.cos(bx*deltac/(f_zeta))*np.exp(-0.5*sw/(f_zeta)**2.)
        fz = f_zeta
    elif side==-1:
        integer1 =math.cos(bx*deltac/(f_zeta*(1-eps*sp(pi, 5))))*np.exp(-0.5*sw/(f_zeta*(1-eps*sp(pi, 5)))**2.)
        fz = f_zeta*(1-eps*sp(pi, 5))
        
    #####################################################################################################
    
    return scipy.integrate.romb(integer1*xgrid**2, dx=dx1)*deltac/(fz*sigmx(m, z, h, wm, pi, side)**2.)



In [ ]:
def di3dm(f_zeta,m, z, h, wm, pi, side):
    eps=0.01
    di2dmr = i3(f_zeta,m*(1+eps), z, h, wm, pi, side)
    di2dml = i3(f_zeta,m*(1-eps), z, h, wm, pi, side)
    return (di2dmr-di2dml)/(2.*eps*m)

In [ ]:
def i4(f_zeta, m, z, h, wm, pi, side):
    factor = 1./(2.*np.pi**2.)
  
    factor1=4*np.pi/vm2(m, z, h, wm, pi, side)
    xgrid = np.linspace(0, rm(m, z, h, wm, pi, side), 2**8+1)
    dx1 = np.diff(xgrid)[0]

    sw, bx, d0dz = sigmaw_and_bx(xgrid, m, z, h, wm, pi, side)
    
    deltac = 1.686*d0dz
    
    ########################################################
    
    if side==1:
        fz = f_zeta*(1+eps*sp(pi, 5))
        integer1 = bx*math.sin(bx*deltac/(fz))*np.exp(-0.5*sw/(fz**2.))
        
    elif side==0:
        fz = f_zeta
        integer1 =bx*math.sin(bx*deltac/(f_zeta))*np.exp(-0.5*sw/fz**2.)
        
    elif side==-1:
        fz = f_zeta*(1-eps*sp(pi, 5))
        integer1 =bx*math.sin(bx*deltac/(f_zeta*(1-eps*sp(pi, 5))))*np.exp(-0.5*sw/(fz)**2.)
        

    
    return scipy.integrate.romb(integer1*xgrid**2, dx=dx1)/(fz**2.)



In [ ]:
def di4dm(f_zeta,m, z, h, wm, pi, side):
    eps=0.01
    di2dmr = i4(f_zeta,m*(1+eps), z, h, wm, pi, side)
    di2dml = i4(f_zeta,m*(1-eps), z, h, wm, pi, side)
    return (di2dmr-di2dml)/(2.*eps*m)


In [ ]:
def halo_bias(zi, m, h, wm, pi, side):
    a= 0.707
    p = 0.3
    dc = 1.686
    nu = dc/smfunc(zi, m, h, wm, pi, side)

    return 1+(a*nu**2-1)/dc+2*p/dc/(1+(a*nu**2)**p)

def halobias_ng(A,f_zeta,k, m, z, h, wm, pi, side):
    
    term1 = halo_bias(z, m, h, wm, pi, side) 
    
    term4 = A/alphak(m, k, z, h, wm, pi, side)*(1-1./dlnnucdm(m, z, h, wm, pi, side)*di3dm(f_zeta,m, z, h, wm, pi, side))
    term5 = (A/alphak(m, k, z, h, wm, pi, side))*(1./dlnnucdm(m, z, h, wm, pi, side))*di4dm(f_zeta,m, z, h, wm, pi, side)
    #print term1, term4, term5
    return term1 + term4 - term5

# Define halo bias

For which we use the Sheth&Tormen, although new fits exist
\begin{equation}
b_{\rm ST} = 1+\frac{a\nu - 1}{\delta_c}+\frac{2p/\delta_c}{1+(a\nu)^p}
\end{equation}
$a=0.707$, and $p=0.3.$ and $\nu = (\delta_c/\sigma)^2$.
Moreover, the window function entering in the fisher matrix is 
\begin{equation}
W_{i(b)}^h(z) = \frac{1}{n_i}\frac{\chi^2(z)}{H(z)}\int{\rm d}M \frac{{\rm d}n}{{\rm d}M}S_{i(b)}(z)b_{\rm ST}(N,z) = 
\frac{1}{n_i}\frac{\chi^2(z)}{H(z)}\bar{W}_{i(b)}^h(z)
\end{equation}



In [ ]:

def window_h_int(m, A, f_zeta, lell,z, zi,  mobs, h, wm, pi, side):
    sigmam = 0.3
    heavy = np.heaviside(z-zbin[zi], 1)*np.heaviside(zbin[zi+1]-z, 1)
    erfi = (special.erfc(x(mobs, m))-special.erfc(x(mobs*(1+np.exp(sigmam)), m)))*1/2
    #integnmean = erfi*halomassfunction(A,f_zeta, zi, m, h, wm, pi, side)*halo_bias(zi, m, h, wm, pi, side)
    integnmean = erfi*halomassfunction(A,f_zeta, zi, m, h, wm, pi, side)*halobias_ng(A,f_zeta,10**lell/dcom(z,h,wm, pi, side), m, zi, h, wm, pi, side)
    return integnmean

#CHECK THIS PLEASE, the delm

def window_h1(A,f_zeta,lell,z, zi, mobs, h, wm, delm, pi, side):
    
    mminint = mobs*10**(-delm)
    mmaxint = mobs*10**(+delm)
    intwind_h = scipy.integrate.quad(window_h_int,  mminint,  mmaxint, args = (A, f_zeta, lell,z, zi, mobs, h, wm, pi, side))[0]
    return intwind_h

window_h = np.vectorize(window_h1)



In [ ]:
#2**8 +1
start = time.time()
print window_h(0.001,1.1,10,0.07, 12, 10**45., href, omegamref, 0.1, 0, 0)
print time.time() - start

# The covariance matrix

The covariance matrix, to be used in the Fisher matrix is 
\begin{equation}
{\rm cov}(N_{i(b)},N_{j(b')}) = N_{i(b)}\delta_{i,j}\delta_{bb'}+S_{i(bb')}\delta_{ij}
\end{equation}
where $i$ and $j$ run over the redshift bins, whereas $b$ and $b'$ over the masses. The function $N_{i(b)}= \Omega_s n_i$, being $\Omega_s $ the survey angle (in deg2). 

The $S_{i(bb')}$ is 
\begin{eqnarray}
S_{i(bb')} &=& \Omega_{s}^2n_i n_j \int\frac{{\rm d}\chi(z)}{\chi^2(z)}W_{i(b)}^h(z) W_{i(b')}^h(z)\int\frac{\ell {\rm d}\ell}{2\pi}\left|\tilde{W}_s(\ell \Theta_s)\right|^2 P\left(k = \frac{\ell}{\chi(z)},z\right)\\
&=& \Omega_{s}^2n_i n_j \int{\rm d}z\frac{\chi^2(z)}{H^2(z)}\frac{1}{n_i n_j}\bar{W}_{i(b)}^h(z) \bar{W}_{i(b')}^h(z)\int\frac{\ell {\rm d}\ell}{2\pi}\left|\tilde{W}_s(\ell \Theta_s)\right|^2 P\left(k = \frac{\ell}{\chi(z)},z\right)
\end{eqnarray}
where 
\begin{equation}
\tilde{W}_s(\ell \Theta_s) = \frac{2J_i(\ell \Theta_s)}{\ell \Theta_s}
\end{equation}
being $J_1$ the spherical Bessel kind and $\Theta_s$ the area of the survey in steradiants  

In [ ]:
def wind_bessel1(ell, thetas):
    spher = scipy.special.spherical_jn(1, ell*thetas, derivative=False)
    return np.abs(2*spher/(ell*thetas))
wind_bessel = np.vectorize(wind_bessel1)

Let us assume for the moment the $n_i$ is mildly dependent on z, hence
\begin{equation}
S_{i(bb')} = \Omega_{s}^2  \int{\rm d}z\frac{\chi^2(z)}{H^2(z)}\bar{W}_{i(b)}^h(z) \bar{W}_{i(b')}^h(z)\int\frac{\ell {\rm d}\ell}{2\pi}\left|\tilde{W}_s(\ell \Theta_s)\right|^2 P\left(k = \frac{\ell}{\chi(z)},z\right)
\end{equation}
Apparently there is no difference: does it mean we can take $n_i$ and $n_j$ out from the calculation? As above?
Or do we need to check what the limit of integration in $z$ from the previous step are?

In [ ]:
def ps_for_ess(zi, k, pi, side):
    if (side==1 and pi==4) or (side==1 and pi==5):
        return intpks(zi,k)
    elif (side==-1 and pi==4) or (side==-1 and pi==5):
        return intpks(zi,k)
    elif side==1: return intpksr(zi,k, pi)
    elif side == 0: return intpks(zi,k)
    elif side == -1: return intpksl(zi,k, pi)
    


#without nmean included
def int_ess(A,f_zeta,z, lell, zi, zj, mobs, mobs1, h, wm, delm, pi, side):
    thetas = 4*np.pi
    omegas = 2000
    winds= window_h(A,f_zeta,z, zi, mobs, h, wm, delm, pi, side)*window_h(A,f_zeta,z, zj, mobs1, h, wm, delm, pi, side)
    volt = (dcom(z, h, wm, pi, side)/hubble(z,h,wm,pi, side))**2
    wind_b = wind_bessel(10**lell, thetas)**2
    pk = ps_for_ess(zi, 10**lell/dcom(z,h,wm, pi, side), pi, side)
    return omegas**2*volt*winds*wind_b*pk*10**(2*lell)/(2*np.pi)




#with nmean included
def int_ess_full(z, lell, zi, zj, mobs, mobs1, h, wm, delm, pi, side,A,f_zeta,nmean1,nmean2):
    
    thetas = 4*np.pi
    omegas = 2000
    winds= window_h(A,f_zeta,lell,z, zi, mobs, h, wm, delm, pi, side)*window_h(A,f_zeta,lell,z, zj, mobs1, h, wm, delm, pi, side)
    volt = (dcom(z, h, wm, pi, side)/hubble(z,h,wm,pi, side))**2
    wind_b = wind_bessel(10**lell, thetas)**2
    pk = ps_for_ess(zi, 10**lell/dcom(z,h,wm, pi, side), pi, side)
    
    ratio_nmean = 1/(nmean1*nmean2)
    
    return ratio_nmean*omegas**2*volt*winds*wind_b*pk*10**(2*lell)/(2*np.pi)

#int_ess_full1 = np.vectorize(int_ess_full)

def integrate2D(dfun, zgrid, lellgrid):
    
    muint = [scipy.integrate.romb(dfun.T[i], dx=np.diff(lellgrid)[0]) for i in range(zgrid.size)]
    return scipy.integrate.romb(muint, dx=np.diff(zgrid)[0])



def ess_func(A, f_zeta,zi, zj, mobs, mobs1, h, wm, delm, pi, side):
    ellmin = np.log10(8)
    ellmax = np.log10(10000)
    zmin = zlist[0]
    zmax = zlist[-1]
    
    zgrid = np.linspace(zmin, zmax, 2**4+1) 
    lellgrid = np.linspace(ellmin, ellmax, 2**4+1)
    z, lell = np.meshgrid(zgrid, lellgrid)

    return integrate2D(int_ess(A, f_zeta,z, lell, zi, zj, mobs, mobs1, h, wm, delm, pi, side),zgrid, lellgrid)
    
    


def ess_func_full(A,f_zeta,zi, zj, mobs, mobs1, h, wm, delm, pi, side):
    ellmin = np.log10(8)
    ellmax = np.log10(10000)
    zmin = zlist[0]
    zmax = zlist[-1]
    zgrid = np.linspace(zmin, zmax, 2**4+1) 
    lellgrid = np.linspace(ellmin, ellmax, 2**4+1)
    z, lell = np.meshgrid(zgrid, lellgrid)
    
    nmean1 = nmean(A,f_zeta,zi, mobs, h, wm, pi, side, delm)
    nmean2 = nmean(A,f_zeta,zj, mobs1, h, wm, pi, side, delm)
    return nmean1*nmean2*integrate2D(int_ess_full(z, lell, zi, zj, mobs, mobs1, h, wm, delm, pi, side,A,f_zeta,nmean1,nmean2),zgrid, lellgrid)
    

Esta es la función que tarda harto: ess_func_full

In [ ]:
#start_time = time.time()
#print ess_func_full(0.01,0.05,0, 0, 10**45., 10**45., 0.67, 0.14, 0.1, 1, 0)
#print(time.time() - start_time)


# Bulding the Cov matrix

###### aqui te da los elementos de la matriz de covarianza, no la matriz completa asi que hay que construirla

In [ ]:
def krone(ii, jj):
    if ii==jj: return 1
    else: return 0


def cov_mat(A,f_zeta, zi, zj, mobs, mobs1, h, wm, delm, pi, side):
    omegas = 2000
    
    if zi==zj:
        s_cov = ess_func_full(A, f_zeta, zi, zj, mobs, mobs1, h, wm, delm, pi, side)

        if mobs==mobs1:
            n_cov=omegas*nmean(A, f_zeta, zi, mobs, h, wm, pi, side, delm)

        elif mobs!=mobs1: 
            n_cov=0
    
    elif zi!=zj:
        s_cov = 0
        n_cov = 0
        
    return n_cov+s_cov
    

In [ ]:
cov_mat1=np.vectorize(cov_mat)

### Creando la matriz de covarianza

In [ ]:
from multiprocessing import Pool
import multiprocessing
from numpy import array

# Loop A and f_zeta

In [ ]:
A1 = [0.01,0.000001, 0.0001]
f_zeta1 = [0.05,0.0001, 0.005]
pickle_files = ['CovMatrix_NG_Af1','CovMatrix_NG_Af2','CovMatrix_NG_Af3']

In [ ]:

def fullmatrix_perz(zi):
    start_time = time.time()

    values=[]
    print('zi',zi)
    for j in range(len(binm)):
        b=binm[j]
        a1=np.tile(binz[0],len(binm))
        a1=list(a1)
        print(j,'b = ',b)
        cov1=cov_mat1(A,f_zeta,zi,a1,b,binm, href, omegamh2ref, 0.1, 8, 0)
        for i in range(1,len(binz)):
            a1=list(np.tile(binz[i],len(binm)))
            cov2=cov_mat1(A,f_zeta,zi,a1,b,binm, href, omegamh2ref, 0.1, 8, 0)
            cov1=np.hstack((cov1,cov2))
        values.append(cov1)

    print(time.time()-start_time)
    return values

In [ ]:
expmass=np.arange(44.2,45.2,0.2)

binm=10**expmass
binz=[0,1,2,3,4,5,6,7,8,9,10,11,12]
largoz=len(binz)
largom=len(binm)

parin = ['wb','h', 'wm', 'ns','a','fz']
nparams=len(parin)
indices_pi = [0, 1, 2, 3, 4, 5]
delm=0.1


for w in range(len(A1)):
    
    ### Cov Matrix ###
    
    A = A1[w]
    f_zeta = f_zeta1[w]
    
    print('Valores A = ', A, 'y f = ', f_zeta)
    p = multiprocessing.Pool()

    resultado = (p.map(fullmatrix_perz,binz))

    matriz_cov=[]
    for i in resultado:
        for j in i:
            matriz_cov.append(j)
            
    matrix_cov_matrix = np.matrix(matriz_cov)
    covmatrix = pd.DataFrame(matrix_cov_matrix)
    
    ### guardando el resultado de la matrix ###
    
    file_pi = open(pickle_files[w]+'.obj', 'w') 
    pickle.dump(covmatrix, file_pi)
    file_pi.close()
    
    ## Visualizando ##
    
    plt.imshow(matriz_cov)
    plt.colorbar()
    plt.show()
    
    # Desde aca todo es muy rapido, maximo 4 minutos
    
    inversa_cov = inv(matrix_cov_matrix)
    
    # formato para que sea easy to read
    
    num=largom
    rangos = np.arange(0,largoz*largom,largom)
    
    new_inverted=[]

    inversa_splitted = np.split(np.array(inversa_cov),largoz)

    for i in range(largoz):
        lista_por_line=[]
        for j in rangos:
            matrix1 = np.array(inversa_splitted[i])[:,j:j+num]
            lista_por_line.append(matrix1)
        new_inverted.append(lista_por_line)
        
    cosmpar = [omegabh2ref, href, omegamh2ref, nsref,A, f_zeta]
    
    # Generando los N right and left para Fisher
    
    nmeanR = []
    nmeanL = []

    for pi in range(len(parin)):
        for m1 in binm:
            nmeanR.append(nmean(A,f_zeta,range(13), m1, href, omegamh2ref, pi, 1, 0.1))
            nmeanL.append(nmean(A,f_zeta,range(13), m1, href, omegamh2ref, pi, -1, 0.1))
            
    nmeanright = []
    nmeanleft = []
    j=0
    for i in range(1,7):
        right = []
        left = []

        for w in binz:
            cosa = np.array(nmeanR[j:5*i])[:,w]
            cosa2 = np.array(nmeanL[j:5*i])[:,w]
            right.append(cosa)
            left.append(cosa2)
        nmeanright.append(right)
        nmeanleft.append(left)
    
        j=j+5
        
    ### Ahora Fisher ###
    
    fishermatrix=[]

    for i in range(nparams):
        print('Indice, parametro y nombre del parametro: ', indices_pi[i],cosmpar[i], parin[i])
        matrix2=[]
        for j in range(nparams):
            suma=0
            for zi in range(len(binz)):
                for zj in range(len(binz)):
                    ni_p = nmeanright[i][zi]
                    ni_m = nmeanleft[i][zi]
 
                    invcov = new_inverted[zi][zj]
                
                    nj_p = nmeanright[j][zj]
                    nj_m = nmeanleft[j][zj]

                    derivative =  (ni_p - ni_m)/(2*eps*cosmpar[i])
                    derivative2 = (nj_p - nj_m)/(2*eps*cosmpar[j])
                
                    derivative = np.matrix(derivative)
                    derivative2 =np.matrix(derivative2)
                
                    fish = np.dot(derivative,np.dot(invcov,derivative2.T))
                
                    suma=suma+fish
  
            matrix2.append(suma)
        fishermatrix.append(matrix2)

    df = pd.DataFrame(fishermatrix, columns = parin, index=parin)
    df1 = df.astype(float)
    arrayfisher=np.matrix(df1)

    inversafisher= arrayfisher.I

    df2 = pd.DataFrame(inversafisher, columns = parin, index=parin)
    for i in parin:
        print('Results')
        print(i, ' = ',np.sqrt(df2[i][i]))

## Errors 68% N-Counts Alone

In [ ]:
#('wb', ' = ', 0.031964234157567946)
#('h', ' = ', 0.1572321302593544)
#('wm', ' = ', 0.15584643336168957)
#('ns', ' = ', 0.5148361299826929)


# References

Primordial power spectrum

https://classylss.readthedocs.io/en/stable/api/classylss.binding.html#classylss.binding.Primordial.get_pk 
ecc 2: https://arxiv.org/pdf/1303.5076.pdf

Transfer function

https://classylss.readthedocs.io/en/stable/api/classylss.binding.html#classylss.binding.Spectra.get_transfer 
https://gitlab.dur.scotgrid.ac.uk/dm-interactions/class_v2.7_ndm/blob/c15781e82037ad55abccd9914171b18194ef638a/python/classy.pyx 
plots: https://ned.ipac.caltech.edu/level5/Sept03/Peacock/Peacock2_5.html

D(z)

https://gitlab.dur.scotgrid.ac.uk/dm-interactions/class_v2.7_ndm/blob/c15781e82037ad55abccd9914171b18194ef638a/python/classy.pyx